In [73]:
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [74]:
import numpy as np 
import pandas as pd

In [75]:
game = pd.read_csv('game.csv')
line_score = pd.read_csv('line_score.csv')
other_stats = pd.read_csv('other_stats.csv')

print("game shape:", game.shape)
print("line_score shape:", line_score.shape)
print("other_stats shape:", other_stats.shape)

print("game columns:", game.columns.tolist())
print("line_score columns:", line_score.columns.tolist())
print("other_stats columns:", other_stats.columns.tolist())

merged_df = game.merge(line_score, on='game_id').merge(other_stats, on='game_id')
print("merged_df shape:", merged_df.shape)
print("merged_df columns:", merged_df.columns.tolist())

merged_df.columns
selected_columns = [
    "season_id", "game_id", "game_date", "team_id_home", "team_name_home",
    "team_abbreviation_home", 
    "team_id_away", "team_abbreviation_away", "team_name_away",
    "season_type", "game_sequence", "pts_qtr1_home", "pts_qtr2_home",
    "pts_qtr3_home", "pts_qtr4_home",
    "pts_qtr1_away", "pts_qtr2_away", "pts_qtr3_away", "pts_qtr4_away",
]
new_df = merged_df[selected_columns]
new_df['game_date'] = pd.to_datetime(new_df['game_date'])

new_df = new_df[new_df['game_date'].dt.year >= 2010]


game shape: (65698, 55)
line_score shape: (58053, 43)
other_stats shape: (28271, 26)
game columns: ['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home', 'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home', 'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away', 'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away', 'season_type']
line_score columns: ['game_date_est', 'game_sequence', 'game_id', 'team_id_home', 'team_abbreviation_home', 'team_city_name

/var/folders/9q/xt_sy4sj7dj02w1x57lg01b16zq9b7/T/ipykernel_35622/4269287426.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['game_date'] = pd.to_datetime(new_df['game_date'])


In [76]:
display(new_df.describe())
display(new_df.info())

# Проверка на наличие нулевых значений в данных
print("Количество нулевых значений в каждом столбце:")
print(new_df.isnull().sum())

# Проверка на наличие выбросов с помощью метода межквартильного размаха (IQR)
def detect_outliers_iqr(df, columns):
    outlier_indices = {}
    for col in columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            outlier_indices[col] = outliers.index.tolist()
            print(f"Столбец '{col}': найдено выбросов - {len(outliers)}")
    return outlier_indices

num_columns = new_df.select_dtypes(include='number').columns.tolist()
outlier_info = detect_outliers_iqr(new_df, num_columns)
new_df = new_df.sort_values(by="game_date", ascending=True)

new_df = new_df[new_df['season_type'].isin(['Regular Season', 'Playoffs'])]



,season_id,game_id,game_date,team_id_home,team_id_away,game_sequence,pts_qtr1_home,pts_qtr2_home,pts_qtr3_home,pts_qtr4_home,pts_qtr1_away,pts_qtr2_away,pts_qtr3_away,pts_qtr4_away
count,14619.000000,1.461900e+04,14619,1.461900e+04,1.461900e+04,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000,14619.000000
mean,22851.098092,2.242413e+07,2016-12-14 11:34:26.324645888,1.606757e+09,1.608299e+09,4.582598,26.622546,26.599631,26.143854,25.660442,26.453041,26.504959,26.105958,25.542582
min,12010.000000,1.100000e+07,2010-01-01 00:00:00,4.100000e+01,4.100000e+01,1.000000,7.000000,6.000000,6.000000,0.000000,7.000000,7.000000,5.000000,0.000000
25%,22013.000000,2.130034e+07,2014-01-15 00:00:00,1.610613e+09,1.610613e+09,2.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,21.000000
50%,22016.000000,2.160092e+07,2017-01-05 00:00:00,1.610613e+09,1.610613e+09,4.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,25.000000
75%,22020.000000,2.200049e+07,2020-02-25 00:00:00,1.610613e+09,1.610613e+09,7.000000,31.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,29.000000
max,42022.000000,4.220040e+07,2023-06-09 00:00:00,1.610617e+09,1.610617e+09,15.000000,55.000000,52.000000,53.000000,51.000000,53.000000,60.000000,59.000000,51.000000
std,5721.349172,5.735414e+06,NaN,7.871508e+07,6.100173e+07,2.940638,6.076446,5.987226,6.085722,5.985608,6.071419,6.039390,6.165451,5.963956


<class 'pandas.core.frame.DataFrame'>
Index: 14619 entries, 13715 to 28333
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   season_id               14619 non-null  int64         
 1   game_id                 14619 non-null  int64         
 2   game_date               14619 non-null  datetime64[ns]
 3   team_id_home            14619 non-null  int64         
 4   team_name_home          14619 non-null  object        
 5   team_abbreviation_home  14619 non-null  object        
 6   team_id_away            14619 non-null  int64         
 7   team_abbreviation_away  14619 non-null  object        
 8   team_name_away          14619 non-null  object        
 9   season_type             14619 non-null  object        
 10  game_sequence           14619 non-null  float64       
 11  pts_qtr1_home           14619 non-null  float64       
 12  pts_qtr2_home           14619 non-null  float64

None

Количество нулевых значений в каждом столбце:
season_id                 0
game_id                   0
game_date                 0
team_id_home              0
team_name_home            0
team_abbreviation_home    0
team_id_away              0
team_abbreviation_away    0
team_name_away            0
season_type               0
game_sequence             0
pts_qtr1_home             0
pts_qtr2_home             0
pts_qtr3_home             0
pts_qtr4_home             0
pts_qtr1_away             0
pts_qtr2_away             0
pts_qtr3_away             0
pts_qtr4_away             0
dtype: int64
Столбец 'season_id': найдено выбросов - 1896
Столбец 'game_id': найдено выбросов - 1896
Столбец 'team_id_home': найдено выбросов - 98
Столбец 'team_id_away': найдено выбросов - 84
Столбец 'game_sequence': найдено выбросов - 6
Столбец 'pts_qtr1_home': найдено выбросов - 64
Столбец 'pts_qtr2_home': найдено выбросов - 105
Столбец 'pts_qtr3_home': найдено выбросов - 120
Столбец 'pts_qtr4_home': найдено выбросо

In [77]:
result = pd.DataFrame()

result['points_difference'] = new_df['pts_qtr1_home'] + new_df['pts_qtr2_home'] + new_df['pts_qtr3_home'] + new_df['pts_qtr4_home'] - new_df['pts_qtr1_away'] - new_df['pts_qtr2_away'] - new_df['pts_qtr3_away'] - new_df['pts_qtr4_away']

# Заполняем значения формы (последние 5, 10 и 15 матчей, без учета того матч был дома или в гостях)

home_form = []
away_form = []

last_team_matches = {}

result['home_form_5'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['away_form_5'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

result['home_form_10'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['away_form_10'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

result['home_form_15'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['away_form_15'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

for index, row in new_df.iterrows():
    home_team = row['team_id_home']
    away_team = row['team_id_away']

    if home_team not in last_team_matches:
        last_team_matches[home_team] = []

    if away_team not in last_team_matches:
        last_team_matches[away_team] = []

    last_home_team_matches = last_team_matches[home_team]
    last_away_team_matches = last_team_matches[away_team]

    counter_home = 0
    counter_away = 0

    for i in range(min(15, len(last_home_team_matches))):
        if last_home_team_matches[-i] > 0:
            counter_home += 1
        if i == 4:
            result.loc[index, 'home_form_5'] = counter_home
        if i == 9:
            result.loc[index, 'home_form_10'] = counter_home
        if i == 14:
            result.loc[index, 'home_form_15'] = counter_home

    for i in range(min(15, len(last_away_team_matches))):
        if last_away_team_matches[-i] > 0:
            counter_away += 1
        if i == 4:
            result.loc[index, 'away_form_5'] = counter_away
        if i == 9:
            result.loc[index, 'away_form_10'] = counter_away
        if i == 14:
            result.loc[index, 'away_form_15'] = counter_away
    
    last_team_matches[home_team].append(result['points_difference'][index])
    last_team_matches[away_team].append(-result['points_difference'][index])




In [78]:
# Заполняем количество дней отдыха между матчами

result['days_off_home'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['days_off_away'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

last_date_for_team = {}

for index, row in new_df.iterrows():
    home_team = row['team_id_home']
    away_team = row['team_id_away']

    if home_team not in last_date_for_team:
        last_date_for_team[home_team] = row['game_date']
        result.loc[index, 'days_off_home'] = 3
    else:
        result.loc[index, 'days_off_home'] = (row['game_date'] - last_date_for_team[home_team]).days
        last_date_for_team[home_team] = row['game_date']

    if away_team not in last_date_for_team:
        last_date_for_team[away_team] = row['game_date']
        result.loc[index, 'days_off_away'] = 3
    else:
        result.loc[index, 'days_off_away'] = (row['game_date'] - last_date_for_team[away_team]).days
        last_date_for_team[away_team] = row['game_date']

# Чистим от выбросов, берем примерно среднее значение
result.loc[result['days_off_home'] > 20, 'days_off_home'] = 3
result.loc[result['days_off_away'] > 20, 'days_off_away'] = 3

display(result.describe())
display(result.info())
display(result.shape)
        


,points_difference,home_form_5,away_form_5,home_form_10,away_form_10,home_form_15,away_form_15,days_off_home,days_off_away
count,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000
mean,0.404519,2.325948,2.315015,4.658965,4.652624,6.971356,6.957872,2.517784,2.495700
std,14.521141,1.108527,1.125822,1.620791,1.627771,2.080068,2.089960,1.465189,1.428617
min,-73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,-9.000000,2.000000,2.000000,4.000000,4.000000,6.000000,6.000000,2.000000,2.000000
50%,0.000000,2.000000,2.000000,5.000000,5.000000,7.000000,7.000000,2.000000,2.000000
75%,10.000000,3.000000,3.000000,6.000000,6.000000,8.000000,8.000000,3.000000,3.000000
max,56.000000,5.000000,5.000000,10.000000,10.000000,14.000000,14.000000,16.000000,15.000000


<class 'pandas.core.frame.DataFrame'>
Index: 13720 entries, 13715 to 28325
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   points_difference  13720 non-null  float64
 1   home_form_5        13720 non-null  float64
 2   away_form_5        13720 non-null  float64
 3   home_form_10       13720 non-null  float64
 4   away_form_10       13720 non-null  float64
 5   home_form_15       13720 non-null  float64
 6   away_form_15       13720 non-null  float64
 7   days_off_home      13720 non-null  float64
 8   days_off_away      13720 non-null  float64
dtypes: float64(9)
memory usage: 1.6 MB


None

(13720, 9)

In [79]:
result['is_regular_season'] = new_df['season_type'].apply(lambda x: 1 if x == 'Regular Season' else 0)
result['is_playoffs'] = new_df['season_type'].apply(lambda x: 1 if x == 'Playoffs' else 0)

display(result.head())
display(result.shape)

display(result.describe())
display(result.info())

,points_difference,home_form_5,away_form_5,home_form_10,away_form_10,home_form_15,away_form_15,days_off_home,days_off_away,is_regular_season,is_playoffs
13715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1,0
13716,12.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1,0
13728,16.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1,0
13726,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1,0
13725,8.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1,0


(13720, 11)

,points_difference,home_form_5,away_form_5,home_form_10,away_form_10,home_form_15,away_form_15,days_off_home,days_off_away,is_regular_season,is_playoffs
count,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000
mean,0.404519,2.325948,2.315015,4.658965,4.652624,6.971356,6.957872,2.517784,2.495700,0.927332,0.072668
std,14.521141,1.108527,1.125822,1.620791,1.627771,2.080068,2.089960,1.465189,1.428617,0.259600,0.259600
min,-73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,-9.000000,2.000000,2.000000,4.000000,4.000000,6.000000,6.000000,2.000000,2.000000,1.000000,0.000000
50%,0.000000,2.000000,2.000000,5.000000,5.000000,7.000000,7.000000,2.000000,2.000000,1.000000,0.000000
75%,10.000000,3.000000,3.000000,6.000000,6.000000,8.000000,8.000000,3.000000,3.000000,1.000000,0.000000
max,56.000000,5.000000,5.000000,10.000000,10.000000,14.000000,14.000000,16.000000,15.000000,1.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
Index: 13720 entries, 13715 to 28325
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   points_difference  13720 non-null  float64
 1   home_form_5        13720 non-null  float64
 2   away_form_5        13720 non-null  float64
 3   home_form_10       13720 non-null  float64
 4   away_form_10       13720 non-null  float64
 5   home_form_15       13720 non-null  float64
 6   away_form_15       13720 non-null  float64
 7   days_off_home      13720 non-null  float64
 8   days_off_away      13720 non-null  float64
 9   is_regular_season  13720 non-null  int64  
 10  is_playoffs        13720 non-null  int64  
dtypes: float64(9), int64(2)
memory usage: 1.8 MB


None

In [80]:
# Номер матча в сезоне

result['match_number_season_home'] = pd.Series([np.nan]*len(result), index=result.index, dtype='float64')
result['match_number_season_away'] = pd.Series([np.nan]*len(result), index=result.index, dtype='float64')

season_matches_current = {}

for index, row in new_df.iterrows():
    season_id = row['season_id']
    away_team = row['team_id_away']
    home_team = row['team_id_home']

    if (season_id, home_team) not in season_matches_current:
        season_matches_current[(season_id, home_team)] = 0

    if (season_id, away_team) not in season_matches_current:
        season_matches_current[(season_id, away_team)] = 0

    season_matches_current[(season_id, home_team)] += 1
    season_matches_current[(season_id, away_team)] += 1

    result.loc[index, 'match_number_season_home'] = season_matches_current[(season_id, home_team)]
    result.loc[index, 'match_number_season_away'] = season_matches_current[(season_id, away_team)]


display(result.describe())
display(result.info())
display(result.shape)
        


,points_difference,home_form_5,away_form_5,home_form_10,away_form_10,home_form_15,away_form_15,days_off_home,days_off_away,is_regular_season,is_playoffs,match_number_season_home,match_number_season_away
count,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000
mean,0.404519,2.325948,2.315015,4.658965,4.652624,6.971356,6.957872,2.517784,2.495700,0.927332,0.072668,31.703936,31.721501
std,14.521141,1.108527,1.125822,1.620791,1.627771,2.080068,2.089960,1.465189,1.428617,0.259600,0.259600,20.271489,20.248491
min,-73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,-9.000000,2.000000,2.000000,4.000000,4.000000,6.000000,6.000000,2.000000,2.000000,1.000000,0.000000,13.000000,14.000000
50%,0.000000,2.000000,2.000000,5.000000,5.000000,7.000000,7.000000,2.000000,2.000000,1.000000,0.000000,31.000000,31.000000
75%,10.000000,3.000000,3.000000,6.000000,6.000000,8.000000,8.000000,3.000000,3.000000,1.000000,0.000000,49.000000,48.000000
max,56.000000,5.000000,5.000000,10.000000,10.000000,14.000000,14.000000,16.000000,15.000000,1.000000,1.000000,78.000000,79.000000


<class 'pandas.core.frame.DataFrame'>
Index: 13720 entries, 13715 to 28325
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   points_difference         13720 non-null  float64
 1   home_form_5               13720 non-null  float64
 2   away_form_5               13720 non-null  float64
 3   home_form_10              13720 non-null  float64
 4   away_form_10              13720 non-null  float64
 5   home_form_15              13720 non-null  float64
 6   away_form_15              13720 non-null  float64
 7   days_off_home             13720 non-null  float64
 8   days_off_away             13720 non-null  float64
 9   is_regular_season         13720 non-null  int64  
 10  is_playoffs               13720 non-null  int64  
 11  match_number_season_home  13720 non-null  float64
 12  match_number_season_away  13720 non-null  float64
dtypes: float64(11), int64(2)
memory usage: 2.0 MB


None

(13720, 13)

In [81]:
# Заполняем значения личных встреч

personal_matches_history = {}
personal_matches_points_history = {}

result['home_over_away_wins_diff_5'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['home_over_away_wins_diff_10'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

result['home_over_away_points_diff_5'] = pd.Series([0]*len(result), index=result.index, dtype='float64')
result['home_over_away_points_diff_10'] = pd.Series([0]*len(result), index=result.index, dtype='float64')

for index, row in new_df.iterrows():
    home_team = row['team_id_home']
    away_team = row['team_id_away']

    if (home_team, away_team) not in personal_matches_history and (away_team, home_team) not in personal_matches_history:
        personal_matches_history[(home_team, away_team)] = []
        personal_matches_history[(away_team, home_team)] = []
        personal_matches_points_history[(home_team, away_team)] = []
        personal_matches_points_history[(away_team, home_team)] = []

    current_history = personal_matches_history[(home_team, away_team)]
    current_history_points = personal_matches_points_history[(home_team, away_team)]

    current_diff = 0
    current_diff_points = 0
    for i in range(min(10, len(current_history))):
        current_diff += current_history[-i]
        current_diff_points += current_history_points[-i]
        if i == 4:
            result.loc[index, 'home_over_away_wins_diff_5'] = current_diff
            result.loc[index, 'home_over_away_points_diff_5'] = current_diff_points
        if i == 9:
            result.loc[index, 'home_over_away_wins_diff_10'] = current_diff
            result.loc[index, 'home_over_away_points_diff_10'] = current_diff_points
    
    if result['points_difference'][index] > 0:
        home_team_win = 1
    elif result['points_difference'][index] < 0:
        home_team_win = -1
    else:
        home_team_win = 0
    points_diff = result['points_difference'][index]

    personal_matches_history[(home_team, away_team)].append(home_team_win)
    personal_matches_history[(away_team, home_team)].append(-home_team_win)
    personal_matches_points_history[(home_team, away_team)].append(points_diff)
    personal_matches_points_history[(away_team, home_team)].append(-points_diff)

display(result.describe())
display(result.info())
display(result.shape)

,points_difference,home_form_5,away_form_5,home_form_10,away_form_10,home_form_15,away_form_15,days_off_home,days_off_away,is_regular_season,is_playoffs,match_number_season_home,match_number_season_away,home_over_away_wins_diff_5,home_over_away_wins_diff_10,home_over_away_points_diff_5,home_over_away_points_diff_10
count,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000
mean,0.404519,2.325948,2.315015,4.658965,4.652624,6.971356,6.957872,2.517784,2.495700,0.927332,0.072668,31.703936,31.721501,-0.024125,-0.052988,-0.468149,-0.710058
std,14.521141,1.108527,1.125822,1.620791,1.627771,2.080068,2.089960,1.465189,1.428617,0.259600,0.259600,20.271489,20.248491,1.977846,2.505166,29.483072,37.432933
min,-73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,-5.000000,-10.000000,-117.000000,-193.000000
25%,-9.000000,2.000000,2.000000,4.000000,4.000000,6.000000,6.000000,2.000000,2.000000,1.000000,0.000000,13.000000,14.000000,-1.000000,-1.000000,-17.000000,-16.000000
50%,0.000000,2.000000,2.000000,5.000000,5.000000,7.000000,7.000000,2.000000,2.000000,1.000000,0.000000,31.000000,31.000000,0.000000,0.000000,0.000000,0.000000
75%,10.000000,3.000000,3.000000,6.000000,6.000000,8.000000,8.000000,3.000000,3.000000,1.000000,0.000000,49.000000,48.000000,1.000000,1.000000,16.000000,14.000000
max,56.000000,5.000000,5.000000,10.000000,10.000000,14.000000,14.000000,16.000000,15.000000,1.000000,1.000000,78.000000,79.000000,5.000000,10.000000,132.000000,205.000000


<class 'pandas.core.frame.DataFrame'>
Index: 13720 entries, 13715 to 28325
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   points_difference              13720 non-null  float64
 1   home_form_5                    13720 non-null  float64
 2   away_form_5                    13720 non-null  float64
 3   home_form_10                   13720 non-null  float64
 4   away_form_10                   13720 non-null  float64
 5   home_form_15                   13720 non-null  float64
 6   away_form_15                   13720 non-null  float64
 7   days_off_home                  13720 non-null  float64
 8   days_off_away                  13720 non-null  float64
 9   is_regular_season              13720 non-null  int64  
 10  is_playoffs                    13720 non-null  int64  
 11  match_number_season_home       13720 non-null  float64
 12  match_number_season_away       13720 non-null  

None

(13720, 17)

In [83]:
result.to_csv('result.csv', index=False)
